In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import jenkspy
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

In [2]:
#!pip install jenkspy --proxy http://rie.proxy.national.agri:8080

In [3]:
def cascade(generateur, nb_iterations):
    fractal = generateur
    for i in range(nb_iterations):
        fractal = np.kron(fractal, generateur)
    return(fractal)

In [4]:
gen1 = np.array([[.5/8,.5/8,.5/8],[.5/8,.5,.5/8],[.5/8,.5/8,.5/8]])

In [5]:
test = cascade(gen1,5)

In [6]:
test.shape

(729, 729)

In [7]:
mpl.use('svg')  

cmap = mpl.cm.RdYlGn_r
bounds = np.quantile(test, np.arange(0,1,.005))
bounds = jenkspy.jenks_breaks(test.flatten()[np.random.choice(test.flatten().shape[0], 1000, replace=False)], n_classes=5)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
mpl.cm.ScalarMappable(norm=norm, cmap=cmap)


fig, ax = plt.subplots(figsize=(20,20))
plt.subplots_adjust(bottom=0, right=1, top=1, left=0)

ax.set_axis_off();
ax.imshow(cmap(norm(test)))
plt.savefig("multifractal.svg", format='svg', dpi=300)

#### Creation des etapes

In [8]:
x = [0,1,2]
y = [0,1,2]
temp = [np.tile(x, len(y))*3, np.repeat(y, len(x))*3]

In [9]:
nine = np.arange(9)

In [10]:
first_level = pd.DataFrame({'x' : nine % 3 , 'y' : nine // 3, 'i' :  nine +1 })
first_level['Prob'] = '$P_' + first_level.i.astype(str) + '$' 

In [11]:
first_level['indexi'] = first_level.i.astype(str)

In [12]:
size = 3
geometry = [ Polygon ([( x * size , y * size) ,
                       ( x * size + size , y * size) ,
                       ( x * size + size , y * size + size ) ,
                       (x * size , y *size + size ) ])
             for x , y in zip (first_level['x'] , first_level['y']) ]
first_level = gpd.GeoDataFrame(first_level, geometry = geometry , crs='EPSG:3035')

In [13]:
fig, ax = plt.subplots(figsize=(20,20))

first_level.boundary.plot(color='black', figsize=(10,10), linewidth=10, ax=ax)
ax.set_axis_off()
plt.subplots_adjust(bottom=0, right=1, top=1, left=0)

centroid = first_level.copy()
centroid['x'] = centroid.geometry.centroid.x
centroid['y'] = centroid.geometry.centroid.y

for x, y, label in zip(centroid.x, centroid.y, centroid.Prob):
    ax.annotate(label, xy=(x-0.5, y-0.5), xytext=(-2, +5), textcoords="offset points", size=200)
plt.savefig("multifractal1.svg", format='svg', dpi=300)

In [14]:
fig, ax = plt.subplots(figsize=(20,20))

first_level.boundary.plot(color='black', figsize=(10,10), linewidth=10, ax=ax)
#first_level[first_level.i==5].boundary.plot(color='red', ax=ax,linewidth=10)
#first_level[first_level.i==5].plot(color='gray', ax=ax,linewidth=10)
ax.set_axis_off()
plt.subplots_adjust(bottom=0, right=1, top=1, left=0)

centroid = first_level.copy()
centroid['x'] = centroid.geometry.centroid.x
centroid['y'] = centroid.geometry.centroid.y

for x, y, label in zip(centroid.x, centroid.y, centroid.indexi):
    ax.annotate(label, xy=(x-0.5, y-0.5), xytext=(-2, +5), textcoords="offset points", size=200)
plt.savefig("multifractal1a.svg", format='svg', dpi=300)

In [15]:
import matplotlib.image as image
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)
santa = image.imread('santa.png')
imagebox = OffsetImage(santa, zoom = 2.5)


fig, ax = plt.subplots(figsize=(20,20))

first_level.boundary.plot(color='black', figsize=(10,10), linewidth=10, ax=ax)
first_level[first_level.i==5].boundary.plot(color='red', ax=ax,linewidth=10)
first_level[first_level.i==5].plot(color='gray', ax=ax,linewidth=10)
ax.set_axis_off()
plt.subplots_adjust(bottom=0, right=1, top=1, left=0)

centroid = first_level.copy()
centroid['x'] = centroid.geometry.centroid.x
centroid['y'] = centroid.geometry.centroid.y

ab = AnnotationBbox(imagebox, (4.5, 4.5), frameon = False)
ax.add_artist(ab)

#for x, y, label in zip(centroid.x, centroid.y, centroid.indexi):
#    ax.annotate(label, xy=(x-0.5, y-0.5), xytext=(-2, +5), textcoords="offset points", size=200)

plt.savefig("multifractal1b2.svg", format='svg', dpi=300)

In [16]:
first_level['santa'] = [None, None, None, None, "🎅".encode("utf8"), None, None, None, None]

In [17]:
fig, ax = plt.subplots(figsize=(20,20))

first_level.boundary.plot(color='black', figsize=(10,10), linewidth=10, ax=ax)
first_level[first_level.i==5].boundary.plot(color='red', ax=ax,linewidth=10)
first_level[first_level.i==5].plot(color='gray', ax=ax,linewidth=10)
ax.set_axis_off()
plt.subplots_adjust(bottom=0, right=1, top=1, left=0)

centroid = first_level.copy()
centroid['x'] = centroid.geometry.centroid.x
centroid['y'] = centroid.geometry.centroid.y

for x, y, label in zip(centroid.x, centroid.y, centroid.indexi):
    ax.annotate(label, xy=(x-0.5, y-0.5), xytext=(-2, +5), textcoords="offset points", size=200)
    
plt.savefig("multifractal1b2.svg", format='svg', dpi=300)

In [18]:
group1 = first_level[first_level.i.isin([1,2,3,4])].copy()
group1['ag'] = 1
group1 = group1.dissolve(by='ag', as_index=False) 

In [19]:
fig, ax = plt.subplots(figsize=(20,20))

first_level.boundary.plot(color='black', figsize=(10,10), linewidth=10, ax=ax)
first_level[first_level.i.isin([1,2,3,4])].plot(color='gray', ax=ax,linewidth=10)
group1.boundary.plot(color='red', ax=ax,linewidth=10)
first_level[first_level.i==5].plot(color='black', ax=ax,linewidth=10)
ax.set_axis_off()
plt.subplots_adjust(bottom=0, right=1, top=1, left=0)

centroid = first_level.copy()
centroid['x'] = centroid.geometry.centroid.x
centroid['y'] = centroid.geometry.centroid.y

for x, y, label in zip(centroid.x, centroid.y, centroid.indexi):
    ax.annotate(label, xy=(x-0.5, y-0.5), xytext=(-2, +5), textcoords="offset points", size=200)
plt.savefig("multifractal1c.svg", format='svg', dpi=300)

In [20]:
group2 = first_level[first_level.i.isin([1,2])].copy()
group2['ag'] = 1
group2 = group2.dissolve(by='ag', as_index=False) 

In [21]:
fig, ax = plt.subplots(figsize=(20,20))

first_level.boundary.plot(color='black', figsize=(10,10), linewidth=10, ax=ax)
first_level[first_level.i.isin([1,2])].plot(color='gray', ax=ax,linewidth=10)
group2.boundary.plot(color='red', ax=ax,linewidth=10)
first_level[first_level.i.isin([5,6,7,8,9])].plot(color='black', ax=ax,linewidth=10)
ax.set_axis_off()
plt.subplots_adjust(bottom=0, right=1, top=1, left=0)

centroid = first_level.copy()
centroid['x'] = centroid.geometry.centroid.x
centroid['y'] = centroid.geometry.centroid.y

for x, y, label in zip(centroid.x, centroid.y, centroid.indexi):
    ax.annotate(label, xy=(x-0.5, y-0.5), xytext=(-2, +5), textcoords="offset points", size=200)
plt.savefig("multifractal1d.svg", format='svg', dpi=300)

In [22]:
fig, ax = plt.subplots(figsize=(20,20))

first_level.boundary.plot(color='black', figsize=(10,10), linewidth=10, ax=ax)
first_level[first_level.i.isin([3])].plot(color='gray', ax=ax,linewidth=10)
first_level[first_level.i.isin([3])].boundary.plot(color='red', ax=ax,linewidth=10)
first_level[first_level.i.isin([1,2,5,6,7,8,9])].plot(color='black', ax=ax,linewidth=10)
ax.set_axis_off()
plt.subplots_adjust(bottom=0, right=1, top=1, left=0)

centroid = first_level.copy()
centroid['x'] = centroid.geometry.centroid.x
centroid['y'] = centroid.geometry.centroid.y

for x, y, label in zip(centroid.x, centroid.y, centroid.indexi):
    ax.annotate(label, xy=(x-0.5, y-0.5), xytext=(-2, +5), textcoords="offset points", size=200)
plt.savefig("multifractal1e.svg", format='svg', dpi=300)

In [23]:
fig, ax = plt.subplots(figsize=(20,20))
santa = image.imread('santa.png')
imagebox = OffsetImage(santa, zoom = 2.5)
first_level.boundary.plot(color='black', figsize=(10,10), linewidth=10, ax=ax)
first_level[first_level.i.isin([3])].plot(color='gray', ax=ax,linewidth=10)
first_level[first_level.i.isin([3])].boundary.plot(color='red', ax=ax,linewidth=10)
first_level[first_level.i.isin([1,2,5,6,7,8,9])].plot(color='black', ax=ax,linewidth=10)
ax.set_axis_off()
plt.subplots_adjust(bottom=0, right=1, top=1, left=0)

centroid = first_level.copy()
centroid['x'] = centroid.geometry.centroid.x
centroid['y'] = centroid.geometry.centroid.y

for x, y, label in zip(centroid.x, centroid.y, centroid.indexi):
    ax.annotate(label, xy=(x-0.5, y-0.5), xytext=(-2, +5), textcoords="offset points", size=200)

ab = AnnotationBbox(imagebox, (7.5, 1.5), frameon = False)
ax.add_artist(ab)
plt.savefig("multifractal1eb.svg", format='svg', dpi=300)

In [24]:
second_level = pd.DataFrame({'x2' : nine % 3 , 'y2' : nine // 3, 'i2' :  nine +1 })
second_level['Prob'] = '$P_' + second_level.i2.astype(str) + '$' 

In [25]:
second_level = first_level.merge(second_level, how='cross')

In [26]:
second_level['Prob'] = '$P_{' + second_level.i2.astype(str) +  '} . P_' + second_level.i.astype(str) + ' $'

In [27]:
second_level.Prob

0     $P_{1} . P_1 $
1     $P_{2} . P_1 $
2     $P_{3} . P_1 $
3     $P_{4} . P_1 $
4     $P_{5} . P_1 $
           ...      
76    $P_{5} . P_9 $
77    $P_{6} . P_9 $
78    $P_{7} . P_9 $
79    $P_{8} . P_9 $
80    $P_{9} . P_9 $
Name: Prob, Length: 81, dtype: object

In [28]:
size = 1
geometry = [ Polygon ([( x * size + x2*3 , y * size + y2*3) ,
                       ( x * size + size + x2*3 , y * size+y2*3 ) ,
                       ( x * size + size+ x2*3 , y * size + size+ y2*3) ,
                       (x * size+ x2*3 , y *size + size+y2*3 ) ])
             for x , y, x2, y2 in zip (second_level['x'] , second_level['y'], second_level['x2'] , second_level['y2']) ]
second_level = gpd.GeoDataFrame(second_level, geometry = geometry , crs='EPSG:3035')

In [29]:
fig, ax = plt.subplots(figsize=(20,20))

first_level.boundary.plot(color='black', figsize=(10,10), linewidth=10, ax=ax)
ax.set_axis_off()
plt.subplots_adjust(bottom=0, right=1, top=1, left=0)
second_level.boundary.plot(ax=ax, zorder=1, ls = '--', color='black')

centroid = second_level.copy()
centroid['x'] = centroid.geometry.centroid.x
centroid['y'] = centroid.geometry.centroid.y

for x, y, label in zip(centroid.x, centroid.y, centroid.Prob):
    ax.annotate(label, xy=(x-0.35, y-0.1), xytext=(-5, +2), textcoords="offset points", size=45)
plt.savefig("multifractal2.svg", format='svg', dpi=300)


In [30]:
test = cascade(gen1,1)

mpl.use('svg')  

cmap = mpl.cm.RdYlGn_r
bounds = np.quantile(test, np.arange(0,1,.05))
bounds = jenkspy.jenks_breaks(test.flatten(), n_classes=3)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
mpl.cm.ScalarMappable(norm=norm, cmap=cmap)


fig, ax = plt.subplots(figsize=(20,20))
plt.subplots_adjust(bottom=0, right=1, top=1, left=0)

ax.set_axis_off();
ax.imshow(cmap(norm(test)))
plt.savefig("multifractal3.svg", format='svg', dpi=300)

In [31]:
test = cascade(gen1,2)
mpl.use('svg')  

cmap = mpl.cm.RdYlGn_r
#bounds = np.quantile(test, np.arange(0,1,.05))
bounds = jenkspy.jenks_breaks(test.flatten(), n_classes=4)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
mpl.cm.ScalarMappable(norm=norm, cmap=cmap)


fig, ax = plt.subplots(figsize=(20,20))
plt.subplots_adjust(bottom=0, right=1, top=1, left=0)

ax.set_axis_off();
ax.imshow(cmap(norm(test)))
plt.savefig("multifractal4.svg", format='svg', dpi=300)

In [32]:
test = cascade(gen1,3)
mpl.use('svg')  

cmap = mpl.cm.RdYlGn_r
#bounds = np.quantile(test, np.arange(0,1,.05))
bounds = jenkspy.jenks_breaks(test.flatten(), n_classes=5)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
mpl.cm.ScalarMappable(norm=norm, cmap=cmap)


fig, ax = plt.subplots(figsize=(20,20))
plt.subplots_adjust(bottom=0, right=1, top=1, left=0)

ax.set_axis_off();
ax.imshow(cmap(norm(test)))
plt.savefig("multifractal5.svg", format='svg', dpi=300)